In [1]:
import requests
from bs4 import BeautifulSoup
import re #regular expressions

In [2]:
#Sending the request
request=requests.get('https://www.zlatestranky.cz/firmy/rubrika/Restaurace/kraj/Hlavn%C3%AD%20m%C4%9Bsto%20Praha')
print(request.status_code)

200


In [14]:
#Making a soup
soup=BeautifulSoup(request.content) #why use .content instead of .text?
#Extracting restaurant titles
titles_soup=soup.find_all('h3')
titles=[]
for i in titles_soup:
    titles.append(i.find('a').text)

print(titles[:10])

['Repre restaurant', 'Restaurace Nad Ledem', 'SRI LANKAN CURRY HOUSE', 'Restaurace HOOTERS Vodičkova', '1. MEXICKÁ RESTAURACE', 'A.H.W. a.s.', 'Abri s.r.o.', 'ABSOLUT s.r.o.', 'ADAMOVÁ IVA', 'Adeta Service, s.r.o.']


In [8]:
#Extracting the link for each restaurant
links=[]
for i in titles_soup:
    links.append(f'https://www.zlatestranky.cz{i.find("a")["href"]}')

print(links[:10])

['https://www.zlatestranky.cz/profil/H270008',
 'https://www.zlatestranky.cz/profil/H2052564',
 'https://www.zlatestranky.cz/profil/H2028609',
 'https://www.zlatestranky.cz/profil/H2056923',
 'https://www.zlatestranky.cz/profil/H952175',
 'https://www.zlatestranky.cz/profil/T1488202',
 'https://www.zlatestranky.cz/profil/H731735',
 'https://www.zlatestranky.cz/profil/T678796',
 'https://www.zlatestranky.cz/profil/H217233',
 'https://www.zlatestranky.cz/profil/H1909419']

In [69]:
#Request for a single restaurant page

#H270008 #original one
#H1778453 #this rest. has more than one tel. number
#H1391009 #has no link, opening hours, email
request_profil=requests.get('https://www.zlatestranky.cz/profil/H1778453')

In [70]:
#Extracting the district 
profil_soup=BeautifulSoup(request_profil.content)
address=profil_soup.find('address').find('span').text
print(address)
re.search('Praha [0-9]{1,2}',address).group(0)

U hráze 27/125, 100 00 Praha 10-Strašniceokres Hlavní město Praha, Hlavní město Praha


'Praha 10'

In [61]:
#Extracting the ratings
profil_soup.find('span',{'itemprop':'ratingValue'}).text

'0'

In [62]:
#Extracting the number of reviews
profil_soup.find('span',{'itemprop':'reviewCount'}).text

'0'

In [71]:
### Extracting the opening hours ###
opening_hours_table=profil_soup.find('table',{'class':'table table-condensed'}).find_all('td') 
opening_hours_table_text=[] #empty list
#extract only day and hours
for i in opening_hours_table:
    if re.match('Dnes|\n',i.text):
        pass
    else:
        opening_hours_table_text.append(i.text)
print(opening_hours_table_text)
#creating a dictionary (key=day,value=opening hours)
opening_hours_dict={}
for i in range(0,len(opening_hours_table_text),2):
    opening_hours_dict[opening_hours_table_text[i]]=opening_hours_table_text[i+1]
print(opening_hours_dict)


['Po', '10 - 23', 'Út', '10 - 23', 'St', '10 - 23', 'Čt', '10 - 23', 'Pá', '10 - 23', 'So', '10:30 - 23', 'Ne', '10:30 - 23']
{'Po': '10 - 23', 'Út': '10 - 23', 'St': '10 - 23', 'Čt': '10 - 23', 'Pá': '10 - 23', 'So': '10:30 - 23', 'Ne': '10:30 - 23'}


In [48]:
### Calculating the opening time span for each day ###
#Writing a function to convert the time span into a number
def time_span_to_number(time_span):
    hours=[time_span.split()[i] for i in [0,2]] #splitting by space and disregarding "-"
    start_end=[] #empty list for converted starting and ending values
    for i in hours:
        if re.search(':',i): #if the string is not a whole hour, convert it
            splits=i.split(':')
            start_end.append(float(splits[0])+float(splits[1])/60)
        else:
            start_end.append(float(i))
    return round(start_end[1] - start_end[0],2)

time_span_dict={key:time_span_to_number(value) for (key,value) in opening_hours_dict.items()}
print(time_span_dict)

{'Po': 13.0, 'Út': 13.0, 'St': 13.0, 'Čt': 13.0, 'Pá': 13.0, 'So': 12.5, 'Ne': 12.5}


In [ ]:
#requests for all pages !!! NOT TESTED YET !!!
request_list=[]
i=1
while request.status_code == 200:
    request=requests.get(f'https://www.zlatestranky.cz/firmy/rubrika/Restaurace/kraj/Hlavn%C3%AD%20m%C4%9Bsto%20Praha/{i}')
    if request.status_code == 200:
        request_list.append(request.content)
        i += 1
    else:
        print(f'Successfuly requested {i-1} pages')
        

In [49]:
#Extracting the phone number

print(profil_soup.find('td', {'itemprop':'telephone'}).text) #finds only the first telephone

#Finding of all tel. numbers
all_telephones = profil_soup.find_all('td', {'itemprop':'telephone'})
telephone_numbers = [] #empty list, that will contain telephone numbers to restaurant
#alternatively, coul be a dict containing {key=name of number, vale=tel. number} -> to be done later

for i in all_telephones:
    telephone_numbers.append(i.text)

print(telephone_numbers)

+420 775 755 061
['+420 775 755 061', '+420 777 104 374']


In [72]:
#Extracting email adress
print(profil_soup.find('a', {'data-ta':'EmailClick'}).text)

#similiarly as in the case of telephones, could be email_dict={'name of email':'email'}
#but it looks like there usually is only one email, so perhaps redundant

restaurace@ubroucku.com


In [73]:
#Extracting link for the restaurant's web page
#print(profil_soup.find('a', {'data-ta':'LinkClick'})['href']) #simply printing the link

if profil_soup.find('a', {'data-ta':'LinkClick'}) == None: #condition, that is useful when the restaurant has no web
    print(None)
else:
    print(profil_soup.find('a', {'data-ta':'LinkClick'})['href'])


http://www.ubroucku.com
